In [1]:
import matplotlib, os
import numpy as np
import imageio.v3 as imageio
from IPython.display import clear_output
from shared.Render import collect_saved_frames, render_function_point
from shared.TestFunctions import TEST_FUNCTIONS
matplotlib.use('Agg')
np.random.seed(42)

#### The differential evolution algorithm

In [2]:
def differential_evolution(func, rnge, num_individuals=20, iterations=20, f=0.8, cr=0.9):
    # Initialize population uniformly within the search bounds and compute fitness
    population = np.random.uniform(rnge[0], rnge[1], size=(num_individuals, 2))
    values = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, population)
    for idx, point in enumerate(population):
        yield 0, idx, tuple(point)
    # Evolution loop over the requested number of iterations
    for iteration in range(1, iterations + 1):
        new_population = population.copy()
        new_values = values.copy()
        # Mutate, recombine, and select for each target vector
        for i in range(num_individuals):
            candidate_indices = np.delete(np.arange(num_individuals), i)
            r1, r2, r3 = np.random.choice(candidate_indices, 3, replace=False)
            # Mutation: create donor vector
            donor = population[r3] + f * (population[r1] - population[r2])
            donor = np.clip(donor, rnge[0], rnge[1])
            # Crossover: binomial scheme ensures at least one donor gene
            trial = population[i].copy()
            j_rand = np.random.randint(2)
            for j in range(2):
                if np.random.uniform() < cr or j == j_rand:
                    trial[j] = donor[j]
            # Selection: accept the trial if it is not worse than the target
            trial_value = func(tuple(trial))
            if trial_value <= values[i]:
                new_population[i] = trial
                new_values[i] = trial_value
                yield iteration, i, tuple(trial)
        population = new_population
        values = new_values

#### Run the differential evolution algorithm and collect frames for multiple functions

In [ ]:
gif_folder_path = "../gifs/differential_evolution"
os.makedirs(gif_folder_path, exist_ok=True)

In [4]:
for func, params in TEST_FUNCTIONS.items():
    print(f"Test function: {func.__name__}")
    rnge = (params[0], params[1])
    resolution = params[2]
    for iter, idx, point in differential_evolution(func, rnge):
        frame = render_function_point(func, rnge=rnge, resolution=resolution, point=point, iteration=f"{iter} - {idx}")
        imageio.imwrite(os.path.join(gif_folder_path, f"{func.__name__}{iter:08d}{idx:08d}.png"), frame)
    collect_saved_frames(gif_folder_path, f"{str.capitalize(func.__name__)}.gif", fps=5)
clear_output()

#### Visualize the differential evolution algorithm on multiple functions

![Sphere](../gifs/differential_evolution/Sphere.gif)
![Ackley](../gifs/differential_evolution/Ackley.gif)
![Rastrigin](../gifs/differential_evolution/Rastrigin.gif)
![Rosenbrock](../gifs/differential_evolution/Rosenbrock.gif)
![Griewank](../gifs/differential_evolution/Griewank.gif)
![Schwefel](../gifs/differential_evolution/Schwefel.gif)
![Levy](../gifs/differential_evolution/Levy.gif)
![Michalewicz](../gifs/differential_evolution/Michalewicz.gif)
![Zakharov](../gifs/differential_evolution/Zakharov.gif)